In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import pyarrow
import fastparquet
import psycopg2
from sqlalchemy import create_engine
import numpy as np
pd.set_option('display.max_colwidth', None)

In [2]:
def create_rating_dataset(df):
    df['id'] = df.index
    df['vote_count'] = df['vote_count'].astype(int)

    # Get the min and max rating values from the dataset
    min_rating = df['rating'].min()
    max_rating = df['rating'].max()
    rating_range = max_rating - min_rating

    # Initialize an empty list to store the rating dataset
    rating_data = []

    # Loop through each recipe in the dataset
    for index, row in df.iterrows():
        # Get the recipe ID and vote count
        recipe_id = row['id']
        vote_count = row['vote_count']

        # Loop through each vote count for the recipe
        for i in range(vote_count):
            # Calculate the normalized rating for the current vote
            rating = (row['rating'] - min_rating) / rating_range * 5

            # Append the rating data to the list
            rating_data.append({'user_id': i+1, 'recipe_id': recipe_id, 'rating': rating})

    # Convert the rating data to a DataFrame
    rating_df = pd.DataFrame(rating_data)

    return rating_df

def load_dataset_to_parquet(df):
    df.to_parquet('../dataset/ratings.parquet.gzip', compression='gzip')

In [3]:
df = pd.read_csv('../dataset/food_recipes.csv')
rating_df = create_rating_dataset(df)
print('creating parquet...')
load_dataset_to_parquet(rating_df)

creating parquet...


In [9]:
kaki = rating_df.loc[rating_df['recipe_id'] == 7527]
kaki['rating'].sum() / 1841

4.426832005706208